In [ ]:
import numpy as np
from IPython.display import Audio
import scipy.signal as sig
import matplotlib.pyplot as plt
import mat73
import os
import masp as srs
import copy
from os.path import join as pjoin
import librosa as lsa
import soundfile as sf

In [ ]:
def crop_echogram(anechoic_echogram):
    nSrc = anechoic_echogram.shape[0]
    nRec = anechoic_echogram.shape[1]
    nBands = anechoic_echogram.shape[2]
    # Returns the "anechoic" version of an echogram
    # Should keep the receiver directivy
    for src in range(nSrc):
        for rec in range(nRec):
            for band in range(nBands):
                anechoic_echogram[src, rec, band].time = anechoic_echogram[src, rec, band].time[:2]
                anechoic_echogram[src, rec, band].coords = anechoic_echogram[src, rec, band].coords[:2, :]
                anechoic_echogram[src, rec, band].value = anechoic_echogram[src, rec, band].value[:2,:]
                anechoic_echogram[src, rec, band].order = anechoic_echogram[src, rec, band].order[:2,:]
    return anechoic_echogram

In [ ]:
def head_2_ku_ears(head_pos,head_orient):
# based on head pos and orientation, compute coordinates of ears
    ear_distance_ku100=0.0875
    theta = (head_orient[0]) * np.pi / 180
    R_ear = [head_pos[0] - ear_distance_ku100 * np.sin(theta),
              head_pos[1] + ear_distance_ku100 * np.cos(theta), 
              head_pos[2]]
    L_ear = [head_pos[0] + ear_distance_ku100 * np.sin(theta),
              head_pos[1] - ear_distance_ku100 * np.cos(theta), 
              head_pos[2]]
    return [L_ear,R_ear]

In [ ]:
def plot_scene(room_dims,head_pos,head_orient,l_mic_pos,l_src_pos,perspective="xy"):
#   function to plot the designed scene
#   room_dims - dimensions of the room [x,y,z]
#   head_pos - head position [x,y,z]
#   head_orient - [az,el]
#   l_src_pos - list of source positions [[x,y,z],...,[x,y,z]]
#   perspective - which two dimensions to show 
    if perspective=="xy":
        dim1=1
        dim2=0
    elif perspective=="yz":
        dim1=2
        dim2=1
    elif perspective=="xz":
        dim1=2
        dim2=0
    fig = plt.figure()
    ax = fig.add_subplot()
    plt.xlim((0,room_dims[dim1]))
    plt.ylim((0,room_dims[dim2]))
    plt.axvline(head_pos[dim1], color='y') # horizontal lines
    plt.axhline(head_pos[dim2], color='y') # vertical lines
    plt.grid(True)
    # plot sources and receivers
    plt.plot(head_pos[dim1],head_pos[dim2], "o", ms=10, mew=2, color="black")
    # plot ears
    plt.plot(l_mic_pos[0][dim1],l_mic_pos[0][dim2], "o", ms=3, mew=2, color="blue")# left ear in blue
    plt.plot(l_mic_pos[1][dim1],l_mic_pos[1][dim2], "o", ms=3, mew=2, color="red")# right ear in red

    for i,src_pos in enumerate(l_src_pos):
        plt.plot(src_pos[dim1],src_pos[dim2], "o", ms=10, mew=2, color="red")
        plt.annotate(str(i), (src_pos[dim1],src_pos[dim2]))
    # plot head orientation if looking from above 
    if perspective=="xy":
        plt.plot(head_pos[dim1],head_pos[dim2], marker=(1, 1, -head_orient[0]), ms=20, mew=2,color="black")

    ax.set_aspect('equal', adjustable='box')


In [ ]:
decoder_path = pjoin('decoders_ord10', 'Ku100_ALFE_Window_sinEQ_bimag.mat') #10th order BimagLS decoder del KU100 sin HA a 48kHz
decoder = mat73.loadmat(decoder_path)['hnm']
decoder = np.roll(decoder,500,axis=0)
maxlim = 2 # maximum reflection time in seconds. Stop simulating if it goes beyond that time.
ambi_order = 10 # ambisonics order

headC_x = 2.0  
headC_y = 2.0
headC_z = 1.0
headOrient_azi = 0.0
headOrient_ele = 0.0
headC = np.array([headC_x, headC_y, headC_z])
headOrient = np.array([headOrient_azi,headOrient_ele])
src = np.array([[1,	1, 1]]) #speech speaker position following convention:

#              _____    _
#             |     |   |
#             |     |   |
#           x ^     |   | l = r[0]
#             |     |   |
#             |     |   |
#             o---->    -
#                  y
#             |-----|

room = np.array([6, 4, 2.5]) #dimensions
rt60=np.array([0.01])
band_centerfreqs=np.array([1000])
fs_rir = 48000
fs_target = fs_rir

speech, fs_speech = lsa.load('ane_speech.wav', sr=fs_rir)

In [ ]:
def process(src, headC, headOrient, room, rt60, band_centerfreqs, maxlim, ambi_order, fs_rir, decoder, speech):
    mic = np.array(head_2_ku_ears(headC,headOrient)) # we get BiMagLS mic points 
    mic = np.vstack((mic, headC)) # we add the head center microphone for MagLS decoders
    nRec = mic.shape[0]
    nSrc = src.shape[0]
    abs_walls,rt60_true = srs.find_abs_coeffs_from_rt(room, rt60)
    # Small correction for sound absorption coefficients:
    if sum(rt60_true-rt60>0.05*rt60_true)>0 :
        abs_walls,rt60_true = srs.find_abs_coeffs_from_rt(room, rt60_true + abs(rt60-rt60_true))
    # Generally, we simulate up to RT60:
    limits = np.minimum(rt60, maxlim)
    # Compute IRs with MASP at 48k:
    abs_echograms = srs.compute_echograms_sh(room, src, mic, abs_walls, limits, ambi_order, headOrient)
    ane_echograms = crop_echogram(copy.deepcopy(abs_echograms))
    mic_rirs = srs.render_rirs_sh(abs_echograms, band_centerfreqs, fs_rir)/np.sqrt(4*np.pi)
    ane_rirs = srs.render_rirs_sh(ane_echograms, band_centerfreqs, fs_rir)/np.sqrt(4*np.pi)
    # Pad anechoic rirs so we don't loose alignment when convolving
    zeros_to_pad = len(mic_rirs) - len(ane_rirs)
    zeros_to_pad = np.zeros((zeros_to_pad, mic_rirs.shape[1], mic_rirs.shape[2], mic_rirs.shape[3]))
    ane_rirs = np.concatenate((ane_rirs, zeros_to_pad))
    bin_ir = np.array([sig.fftconvolve(np.squeeze(mic_rirs[:,:,0, 0]), decoder[:,:,0], 'full', 0).sum(1),
                    sig.fftconvolve(np.squeeze(mic_rirs[:,:,1, 0]), decoder[:,:,1], 'full', 0).sum(1)])
    bin_aneIR = np.array([sig.fftconvolve(np.squeeze(ane_rirs[:,:,0, 0]), decoder[:,:,0], 'full', 0).sum(1),
                    sig.fftconvolve(np.squeeze(ane_rirs[:,:,1, 0]), decoder[:,:,1], 'full', 0).sum(1)])
    reverberant_src = np.array([sig.fftconvolve(speech, bin_ir[0, :], 'same'), sig.fftconvolve(speech, bin_ir[1, :], 'same')])
    anechoic_src = np.array([sig.fftconvolve(speech, bin_aneIR[0, :], 'same'), sig.fftconvolve(speech, bin_aneIR[1, :], 'same')])
    return reverberant_src, anechoic_src, mic

In [ ]:
headOrient = np.array([0.,0.])
_, deg0, mic0 = process(src, headC, headOrient, room, rt60, band_centerfreqs, maxlim, ambi_order, fs_rir, decoder, speech)
plot_scene(room,headC, headOrient,mic0,src,perspective="xy")
Audio(deg0, rate=fs_rir)

In [ ]:
headOrient = np.array([90.,0.])
_, deg90, mic90 = process(src, headC, headOrient, room, rt60, band_centerfreqs, maxlim, ambi_order, fs_rir, decoder, speech)
plot_scene(room,headC, headOrient,mic90,src,perspective="xy")
Audio(deg90, rate=fs_rir)

In [ ]:
headOrient = np.array([180.,0.])
_, deg180, mic180 = process(src, headC, headOrient, room, rt60, band_centerfreqs, maxlim, ambi_order, fs_rir, decoder, speech)
plot_scene(room,headC, headOrient,mic180,src,perspective="xy")
Audio(deg180, rate=fs_rir)